<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Загрузка-и-просмотр-данных" data-toc-modified-id="Загрузка-и-просмотр-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка и просмотр данных</a></span></li><li><span><a href="#Предобработка-текста" data-toc-modified-id="Предобработка-текста-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Предобработка текста</a></span></li><li><span><a href="#Борьба-с-дисбалансом-классов" data-toc-modified-id="Борьба-с-дисбалансом-классов-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Борьба с дисбалансом классов</a></span></li><li><span><a href="#Векторизация-текста" data-toc-modified-id="Векторизация-текста-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Векторизация текста</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span><ul class="toc-item"><li><span><a href="#Взвешивание-классов" data-toc-modified-id="Взвешивание-классов-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Взвешивание классов</a></span></li><li><span><a href="#Upsample" data-toc-modified-id="Upsample-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Upsample</a></span></li><li><span><a href="#Downsample" data-toc-modified-id="Downsample-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Downsample</a></span></li></ul></li><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Дерево решений</a></span><ul class="toc-item"><li><span><a href="#Взвешивание-классов" data-toc-modified-id="Взвешивание-классов-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Взвешивание классов</a></span></li></ul></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Градиентный-бустинг" data-toc-modified-id="Градиентный-бустинг-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Градиентный бустинг</a></span><ul class="toc-item"><li><span><a href="#LGBMClassifier" data-toc-modified-id="LGBMClassifier-2.4.1"><span class="toc-item-num">2.4.1&nbsp;&nbsp;</span>LGBMClassifier</a></span></li><li><span><a href="#Upsample" data-toc-modified-id="Upsample-2.4.2"><span class="toc-item-num">2.4.2&nbsp;&nbsp;</span>Upsample</a></span></li><li><span><a href="#Downsample" data-toc-modified-id="Downsample-2.4.3"><span class="toc-item-num">2.4.3&nbsp;&nbsp;</span>Downsample</a></span></li></ul></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок. Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import numpy as np
import re
import time

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression

from catboost import CatBoostClassifier
from catboost import Pool
from lightgbm import LGBMClassifier
from sklearn.utils import shuffle

from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords as nltk_stopwords
import nltk

#! pip install spacy
import spacy

import sys
#!{sys.executable} -m pip install spacy
# Download spaCy's  'en' Model
#!{sys.executable} -m spacy download en

In [2]:
# настроим отображение формата float
pd.set_option('display.float_format', '{:.2f}'.format)

# настроим отображение всех столбцов таблиц
pd.set_option('display.max_columns', None)

nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /Users/denis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Загрузка и просмотр данных

In [3]:
try:
    data = pd.read_csv('/Users/anastasia/Documents/YandexPractikum/Спринт_13_МL_для текстов/Проект_13/toxic_comments.csv')
except:
    data = pd.read_csv('./datasets/toxic_comments.csv')
    
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [4]:
data.head(10)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


In [5]:
data['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

После беглого взгляда на данные видим, что перед нами стоит задача бинарной классификации, где 1 - токсичный комментарий, а 0 - приемлемый комментарий. Кроме того, можем отметить сильный дисбаланс классов, который нужно будет учесть в дальнейшем. 

### Предобработка текста

Первоначально разобьем текст на леммы и очистим его от лишних символов. Для этого напишем функцию под каждую из задач.

In [6]:
# функция для лемматизации
load_model = spacy.load('en_core_web_sm', disable = ['parser','ner'])

def lemmatize(text):
    doc = load_model(text.lower())
    lemm_text = " ".join([token.lemma_ for token in doc])
    return lemm_text

In [7]:
# функция, чтобы оставить в тексте только буквенные символы и пробелы
def clear_text(text):
    new_text = re.sub(r'[^a-zA-Z]', ' ', text)
    split_text = new_text.split()
    join_text = " ".join(split_text)
    return join_text

Применим функции к нашему столбцу `text` и создадим новый столбец `lemm_text` с очищенным и лемматизированным текстом. 

In [8]:
%%time

text_c = list(map(clear_text, list(data['text'])))
text_l = list(map(lemmatize, text_c))

CPU times: user 11min 8s, sys: 3.94 s, total: 11min 12s
Wall time: 12min 24s


In [9]:
data['lemm_text'] = text_l

In [10]:
display(data.head())
display(data.tail())

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edit make under my usernam...
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour I m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man I m really not try to edit war it s ju...
3,"""\nMore\nI can't make any real suggestions on ...",0,more I can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...


,text,toxic,lemm_text
159566,""":::::And for the second time of asking, when ...",0,and for the second time of ask when your view ...
159567,You should be ashamed of yourself \n\nThat is ...,0,you should be ashamed of yourself that be a ho...
159568,"Spitzer \n\nUmm, theres no actual article for ...",0,spitzer umm there s no actual article for pros...
159569,And it looks like it was actually you who put ...,0,and it look like it be actually you who put on...
159570,"""\nAnd ... I really don't think you understand...",0,and I really don t think you understand I come...


### Борьба с дисбалансом классов

Как ранее мы выяснили, классы достаточно несбалансированные, поэтому в процессе попробуем разные способы балансировки. Для этого создадим две вспомогательных выборки - увеличивающей наименее часты класс и уменьшающей наиболее частый класс. 

Далее выделим обучающую и тестовую выборки для каждого набора данных:

In [11]:
# создание тарета и признаков
target = data['toxic']
features = data['lemm_text']

In [12]:
# выделим обучающую и тренировочную выборки
target_train, target_test, features_train, features_test = train_test_split(
    target, features, test_size=0.25, random_state = 12345)


print(features_train.shape)
print(target_train.shape)
print(features_test.shape)
print(target_test.shape)

(119678,)
(119678,)
(39893,)
(39893,)


In [13]:
target_train.value_counts()

0    107540
1     12138
Name: toxic, dtype: int64

In [14]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

In [15]:
features_train_up, target_train_up = upsample(features_train, target_train, 8)
target_train_up.value_counts()

0    107540
1     97104
Name: toxic, dtype: int64

In [16]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

In [17]:
features_train_down, target_train_down = downsample(features_train, target_train, 0.12)
target_train_down.value_counts()

0    12905
1    12138
Name: toxic, dtype: int64

### Векторизация текста

Подготовим текстовые данные для использования их в качестве признаков - переведем их в векторный формат. Кроме того удалим стоп-слова. Обучим модель только на тренировочных данных.

In [19]:
# создадим векторное представление текста без N-грамм

count_tf_idf = TfidfVectorizer(stop_words=stopwords)

features_train_tfidf = count_tf_idf.fit_transform(features_train) 
features_test_tfidf = count_tf_idf.transform(features_test) 

# создадим векторное представление текста с N-граммами
count_tfidf_n = TfidfVectorizer(stop_words=stopwords, ngram_range=(2, 2))
features_train_ntfidf = count_tfidf_n.fit_transform(features_train) 
features_test_ntfidf = count_tfidf_n.transform(features_test) 


print(features_train_tfidf.shape)
print(features_test_tfidf.shape)

(119678, 128268)
(39893, 128268)


In [20]:
# создадим векторное представление текста с увеличенной выборкой

features_train_up_tfidf = count_tf_idf.fit_transform(features_train_up)
features_test_up_tfidf = count_tf_idf.transform(features_test) 

print(features_train_up_tfidf.shape)
print(features_test_up_tfidf.shape)

(204644, 128268)
(39893, 128268)


In [21]:
# создадим векторное представление текста с сокращенной выборкой

features_train_down_tfidf = count_tf_idf.fit_transform(features_train_down) 
features_test_down_tfidf = count_tf_idf.transform(features_test) 

print(features_train_down_tfidf.shape)
print(features_test_down_tfidf.shape)

(25043, 46218)
(39893, 46218)


## Обучение

Протестируем разные модели, посмотрим, что будет лучше предсказывать тональность текста.

### Логистическая регрессия

#### Взвешивание классов

In [22]:
# без N-грамм
reg = LogisticRegression(solver='liblinear', class_weight='balanced', random_state = 12345)
reg.fit(features_train_tfidf, target_train)

LogisticRegression(class_weight='balanced', random_state=12345,
                   solver='liblinear')

In [23]:
prediction_reg = reg.predict(features_test_tfidf)

f1_reg = f1_score(target_test, prediction_reg)
recall_reg = recall_score(target_test, prediction_reg)
precision_reg = precision_score(target_test, prediction_reg)

print("f1 на тестовом наборе: {:.4f}".format(f1_reg))
print("recall на тестовом наборе: {:.4f}".format(recall_reg))
print("precision на тестовом наборе: {:.4f}".format(precision_reg))

f1 на тестовом наборе: 0.7544
recall на тестовом наборе: 0.8615
precision на тестовом наборе: 0.6709


In [26]:
# с n-граммами (1, 3)
count_tfidf_1_3 = TfidfVectorizer(stop_words=stopwords, ngram_range=(1, 3))
features_train_tfidf_1_3 = count_tfidf_1_3.fit_transform(features_train) 
features_test_tfidf_1_3 = count_tfidf_1_3.transform(features_test) 

print(features_train_tfidf_1_3.shape)
print(features_test_tfidf_1_3.shape)

reg_n = LogisticRegression(solver='liblinear', class_weight='balanced', random_state = 12345, C=20)
reg_n.fit(features_train_tfidf_1_3, target_train)

(119678, 5122048)
(39893, 5122048)


LogisticRegression(C=20, class_weight='balanced', random_state=12345,
                   solver='liblinear')

In [27]:
# с n-граммами (1, 3)
prediction_reg_n_1_3 = reg_n.predict(features_test_tfidf_1_3)

f1_reg_n_1_3 = f1_score(target_test, prediction_reg_n_1_3)
recall_reg_n_1_3 = recall_score(target_test, prediction_reg_n_1_3)
precision_reg_n_1_3 = precision_score(target_test, prediction_reg_n_1_3)

print("f1 на тестовом наборе: {:.4f}".format(f1_reg_n_1_3))
print("recall на тестовом наборе: {:.4f}".format(recall_reg_n_1_3))
print("precision на тестовом наборе: {:.4f}".format(precision_reg_n_1_3))

f1 на тестовом наборе: 0.7904
recall на тестовом наборе: 0.8079
precision на тестовом наборе: 0.7737


#### Upsample

In [28]:
reg_up = LogisticRegression(solver='sag', random_state = 12345)
reg_up.fit(features_train_up_tfidf, target_train_up)

LogisticRegression(random_state=12345, solver='sag')

In [29]:
prediction_reg_up = reg_up.predict(features_test_up_tfidf)

f1_reg_up = f1_score(target_test, prediction_reg_up)
recall_reg_up = recall_score(target_test, prediction_reg_up)
precision_reg_up = precision_score(target_test, prediction_reg_up)

print("f1 на тестовом наборе: {:.4f}".format(f1_reg_up))
print("recall на тестовом наборе: {:.4f}".format(recall_reg_up))
print("precision на тестовом наборе: {:.4f}".format(precision_reg_up))

f1 на тестовом наборе: 0.7605
recall на тестовом наборе: 0.8429
precision на тестовом наборе: 0.6927


#### Downsample

In [30]:
reg_down = LogisticRegression(solver='liblinear', random_state = 12345)
reg_down.fit(features_train_down_tfidf, target_train_down)

LogisticRegression(random_state=12345, solver='liblinear')

In [31]:
prediction_reg_down = reg_down.predict(features_test_down_tfidf)

f1_reg_down = f1_score(target_test, prediction_reg_down)
recall_reg_down = recall_score(target_test, prediction_reg_down)
precision_reg_down = precision_score(target_test, prediction_reg_down)

print("f1 на тестовом наборе: {:.4f}".format(f1_reg_down))
print("recall на тестовом наборе: {:.4f}".format(recall_reg_down))
print("precision на тестовом наборе: {:.4f}".format(precision_reg_down))

f1 на тестовом наборе: 0.6980
recall на тестовом наборе: 0.8608
precision на тестовом наборе: 0.5870


### Дерево решений

#### Взвешивание классов

In [32]:
# подберем гиперпараметры для модели дерева решений
param_grid_tree = {'max_depth':range(1, 11, 1)}

grid_tree = GridSearchCV(
    DecisionTreeClassifier(class_weight='balanced', random_state = 12345), 
    param_grid_tree, cv=5, scoring='f1',
    n_jobs=-1)

grid_tree.fit(features_train_tfidf, target_train)

GridSearchCV(cv=5,
             estimator=DecisionTreeClassifier(class_weight='balanced',
                                              random_state=12345),
             n_jobs=-1, param_grid={'max_depth': range(1, 11)}, scoring='f1')

In [33]:
prediction_tree = grid_tree.predict(features_test_tfidf)

f1_tree = f1_score(target_test, prediction_tree)
recall_tree = recall_score(target_test, prediction_tree)
precision_tree = precision_score(target_test, prediction_tree)

print("Наилучшие параметры: {}".format(grid_tree.best_params_))
print("Наилучшее значение f1 при перекрестной проверке:{:.2f}".format(grid_tree.best_score_))
print("f1 на тестовом наборе: {:.4f}".format(f1_tree))
print("recall на тестовом наборе: {:.4f}".format(recall_tree))
print("precision на тестовом наборе: {:.4f}".format(precision_tree))

Наилучшие параметры: {'max_depth': 9}
Наилучшее значение f1 при перекрестной проверке:0.57
f1 на тестовом наборе: 0.5689
recall на тестовом наборе: 0.4253
precision на тестовом наборе: 0.8591


### Случайный лес

In [34]:
param_grid_forest = {'max_depth':range(20, 61, 10), 'n_estimators':range(10, 81, 10)}

grid_forest = GridSearchCV(
    RandomForestClassifier(class_weight='balanced', random_state = 12345), 
    param_grid_forest, cv=5, scoring='f1',
    n_jobs=-1)

grid_forest.fit(features_train_tfidf, target_train)

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(class_weight='balanced',
                                              random_state=12345),
             n_jobs=-1,
             param_grid={'max_depth': range(20, 61, 10),
                         'n_estimators': range(10, 81, 10)},
             scoring='f1')

In [35]:
prediction_forest = grid_forest.predict(features_test_tfidf)

f1_forest = f1_score(target_test, prediction_forest)
recall_forest = recall_score(target_test, prediction_forest)
precision_forest = precision_score(target_test, prediction_forest)

print("Наилучшие параметры: {}".format(grid_forest.best_params_))
print("Наилучшее значение f1 при перекрестной проверке:{:.4f}".format(grid_forest.best_score_))
print("f1 на тестовом наборе: {:.4f}".format(f1_forest))
print("recall на тестовом наборе: {:.4f}".format(recall_forest))
print("precision на тестовом наборе: {:.4f}".format(precision_forest))

Наилучшие параметры: {'max_depth': 60, 'n_estimators': 80}
Наилучшее значение f1 при перекрестной проверке:0.4926
f1 на тестовом наборе: 0.4933
recall на тестовом наборе: 0.7563
precision на тестовом наборе: 0.3660


### Градиентный бустинг

#### LGBMClassifier

In [37]:
param_grid_lgbm = {'depth':list(range(1, 10, 2)), 'learning_rate':[0.1, 0.5],
                 'iterations':[500, 1000]}

grid_lgbm = GridSearchCV(
    LGBMClassifier(loss_function='f1', verbose=500, class_weight='balanced'), param_grid_lgbm, cv=5, scoring='f1',
    n_jobs=-1)

grid_lgbm.fit(features_train_tfidf, target_train)

[LightGBM] [Warning] Unknown parameter: depth
[LightGBM] [Warning] Unknown parameter: loss_function
[LightGBM] [Warning] Unknown parameter: iterations
[LightGBM] [Warning] Unknown parameter: depth
[LightGBM] [Warning] Unknown parameter: loss_function
[LightGBM] [Warning] Unknown parameter: iterations
[LightGBM] [Info] Number of positive: 12138, number of negative: 107540
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.997478
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.997478
[LightGBM] [Debug] init for col-wise cost 0.958196 seconds, init for row-wise cost 1.138799 seconds
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 1.301052 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 517329
[LightGBM] [Info] Number of data points in the train set: 119678, number of used features: 9472
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.

GridSearchCV(cv=5,
             estimator=LGBMClassifier(class_weight='balanced',
                                      loss_function='f1', verbose=500),
             n_jobs=-1,
             param_grid={'depth': [1, 3, 5, 7, 9], 'iterations': [500, 1000],
                         'learning_rate': [0.1, 0.5]},
             scoring='f1')

In [38]:
prediction_lgbm = grid_lgbm.predict(features_test_tfidf)

f1_lgbm = f1_score(target_test, prediction_lgbm)
recall_lgbm = recall_score(target_test, prediction_lgbm)
precision_lgbm = precision_score(target_test, prediction_lgbm)


print("Наилучшие параметры: {}".format(grid_lgbm.best_params_))
print("Наилучшее значение f1 при перекрестной проверке:{:.2f}".format(grid_lgbm.best_score_))
print("f1 на тестовом наборе: {:.4f}".format(f1_lgbm))
print("recall на тестовом наборе: {:.4f}".format(recall_lgbm))
print("precision на тестовом наборе: {:.4f}".format(precision_lgbm))

Наилучшие параметры: {'depth': 1, 'iterations': 500, 'learning_rate': 0.5}
Наилучшее значение f1 при перекрестной проверке:0.75
f1 на тестовом наборе: 0.7517
recall на тестовом наборе: 0.8314
precision на тестовом наборе: 0.6859


#### Upsample

In [39]:
param_grid_lgbm = {'depth':list(range(1, 10, 2)), 'learning_rate':[0.1, 0.5],
                 'iterations':[500, 1000]}

grid_lgbm = GridSearchCV(
    LGBMClassifier(loss_function='f1', verbose=1000), param_grid_lgbm, cv=5, scoring='f1')

grid_lgbm.fit(features_train_up_tfidf, target_train_up)

[LightGBM] [Warning] Unknown parameter: depth
[LightGBM] [Warning] Unknown parameter: loss_function
[LightGBM] [Warning] Unknown parameter: iterations
[LightGBM] [Warning] Unknown parameter: depth
[LightGBM] [Warning] Unknown parameter: loss_function
[LightGBM] [Warning] Unknown parameter: iterations
[LightGBM] [Info] Number of positive: 77683, number of negative: 86032
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.998131
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.998131
[LightGBM] [Debug] init for col-wise cost 1.443058 seconds, init for row-wise cost 1.608634 seconds
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 1.783788 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 525065
[LightGBM] [Info] Number of data points in the train set: 163715, number of used features: 11222
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.474501 -> initscore=-0

KeyboardInterrupt: 

In [ ]:
prediction_lgbm_up = grid_lgbm.predict(features_test_up_tfidf)

f1_lgbm_up = f1_score(target_test, prediction_lgbm_up)
recall_lgbm_up = recall_score(target_test, prediction_lgbm_up)
precision_lgbm_up = precision_score(target_test, prediction_lgbm_up)


print("Наилучшие параметры: {}".format(grid_lgbm.best_params_))
print("Наилучшее значение f1 при перекрестной проверке:{:.2f}".format(grid_lgbm.best_score_))
print("f1 на тестовом наборе: {:.2f}".format(f1_lgbm_up))
print("recall на тестовом наборе: {:.2f}".format(recall_lgbm_up))
print("precision на тестовом наборе: {:.2f}".format(precision_lgbm_up))

#### Downsample

In [ ]:
param_grid_lgbm = {'depth':list(range(1, 10, 2)), 'learning_rate':[0.1, 0.5],
                 'iterations':[500, 1000]}

grid_lgbm_down = GridSearchCV(
    LGBMClassifier(loss_function='f1', verbose=1000), param_grid_lgbm, cv=5, scoring='f1')

grid_lgbm_down.fit(features_train_down_tfidf, target_train_down)

In [ ]:
prediction_lgbm_down = grid_lgbm_down.predict(features_test_down_tfidf)

f1_lgbm_down = f1_score(target_test, prediction_lgbm_down)
recall_lgbm_down = recall_score(target_test, prediction_lgbm_down)
precision_lgbm_down = precision_score(target_test, prediction_lgbm_down)


print("Наилучшие параметры: {}".format(grid_lgbm.best_params_))
print("Наилучшее значение f1 при перекрестной проверке:{:.2f}".format(grid_lgbm.best_score_))
print("f1 на тестовом наборе: {:.2f}".format(f1_lgbm_down))
print("recall на тестовом наборе: {:.2f}".format(recall_lgbm_down))
print("precision на тестовом наборе: {:.2f}".format(precision_lgbm_down))

## Выводы

In [41]:
# настроим отображение формата float
pd.set_option('display.float_format', '{:.4f}'.format)

In [43]:
table = pd.DataFrame(data = [[f1_reg_n_1_3, recall_reg_n_1_3, precision_reg_n_1_3],
                             [f1_reg_up, recall_reg_up, precision_reg_up],
                             [f1_tree, recall_tree, precision_tree], 
                             [f1_forest, recall_forest, precision_forest],
                             [f1_lgbm, recall_lgbm, precision_lgbm]],
                             columns = ['F1', 'Recall', 'Precision'],
                             index = ['LogReg с n-грамм (1, 3)', 'LogReg', 'DecisionTree', 'RandomForest', 'LightGBM'])

table

,F1,Recall,Precision
"LogReg с n-грамм (1, 3)",0.7904,0.8079,0.7737
LogReg,0.7605,0.8429,0.6927
DecisionTree,0.5689,0.4253,0.8591
RandomForest,0.4933,0.7563,0.3660
LightGBM,0.7517,0.8314,0.6859


**Вывод**

В ходе проекта были предложены модели, позволяющие обрабатывать текстовую информацию и искать токсичные комментарии и отправлять их на модерацию. Для этого была проведена лемматизация теста, его очистка от лишних символов и стоп слов. Кроме того, из-за несбалансированности категорий, для ряда моделей (не для всех, в целях экономии времени, но для тех, что с большей вероятностью дадут лучший результат) были применены методы борьбы с дисбалансом - взвешивание классов, upsample, downsample. Наконец, для того, чтобы можно было считать текстовые данные как признаки, они были приведены к векторному виду с помощью матрицы значений TF-IDF.

На векторизованных данных были обучены модели `Decision Tree`, `Random Forest`, `Gradient Boosting` и `Logistic Regression`. Моделью, показавшей наибольшую метрику качества F1 (и больше 0.75) стала модель `Logistic Regression` с увеличенной выборкой (upsample), где значение f1 - 0.76 и с n-граммами, f1 - 0.79. Данные модели позволяют наиболее точно определить токсичные комментарии, чтобы вдальнейшем их отсеить. 